In [ ]:
# code with detailed comments

In [1]:
import boto3
from datetime import datetime

In [4]:
today = datetime.today()

In [5]:
today

datetime.datetime(2024, 2, 5, 10, 45, 57, 436357)

In [6]:
todays_date = today.strftime("%Y%m%d")

In [7]:
todays_date

'20240205'

In [8]:
# connect to s3 bucket with boto3
s3_client = boto3.client('s3')

In [9]:
# set a variable to our bucket's name in aws
bucket_name = "nol-organize-s3-objects"

In [ ]:
# list all objects in our s3 bucket (using version 2)
list_objects_response = s3_client.list_objects_v2(
    Bucket=bucket_name
)

In [ ]:
list_objects_response

In [ ]:
get_contents = list_objects_response.get("Contents")

In [ ]:
get_contents # it's a list of items

In [ ]:
# get all key's names from the list and store them in a variable
get_all_s3_object_and_folder_names = []

# loop through the list and collect the values of keys

for i in get_contents:
    s3_object_name = i.get("Key")
    
    get_all_s3_object_and_folder_names.append(s3_object_name)

In [ ]:
get_all_s3_object_and_folder_names

In [ ]:
# Create a folder in the s3 bucket 
# Folder must be created according to today's date
# Hence every day we shall have a new folder according to the actual day
# In aws, the name of the folder finishes with a /

directory_name = todays_date + "/"

In [ ]:
directory_name

In [ ]:
# Check whether today's folder exists in s3 bucket, using an IF statement
# If not, create a folder for today's date
# the folders' names are store in a list
# So check if the items match the names in the list -> get_all_s3_object_and_folder_names

if directory_name not in get_all_s3_object_and_folder_names:
    s3_client.put_object(
        Bucket=bucket_name,
        Key=(directory_name)
    )



In [ ]:
# Add any objects that we've uploaded today into the "Today's floder"
# Two conditions: if the "last modified's date" of the object matches the "Today's floder" name
# If the objects we wanna move into the "Today's floder" don't have a slash at the end of their names,
# Because it's only the "Today's floder" which has a /, and we don't wanna move those folder into another folder
# the / is a knid of differenciator between the "Today's floder" and the files we wanna move

# In aws there's no integrated "function" for moving
# So create a function ton copy the file
# And another one to delete the file that has been copied

for i in get_contents:
    object_creation_date = i.get("LastModified").strftime("%Y%m%d") + "/"
    object_name = i.get("Key")
    
    if object_creation_date == directory_name and "/" not in object_name:
        s3_client.copy_object(
            Bucket=bucket_name,
            CopySource=bucket_name+"/"+object_name, # path to the object in s3
            Key=directory_name+object_name # in S3 when copying we can give a new name, in our example we keet the actual object name we're copying
        )
        
        s3_client.delete_object(
            Bucket=bucket_name,
            Key=object_name
        )